# YOLOv8 License Plate Detection

In [1]:
# missing yolo dep
!pip install lapx>=0.5.2

In [3]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.6/911.6 kB 44.1 MB/s eta 0:00:00


In [4]:
import cv2 as cv
from glob import glob
import os
import random
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Pre-Trained YOLOv8

The regular YOLOv8 training weights do not contain a `number_plate` class and cannot be used directly for a number plate detection:

In [5]:
# read in video paths
videos = glob('inputs/*.mp4')
print(videos)

[]


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# pick pre-trained model
model_pretrained = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 317MB/s]


In [8]:
# read video by index
video = cv.VideoCapture(videos[1])

# get video dims
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'DIVX')
out = cv.VideoWriter('./outputs/uk_dash_2.avi', fourcc, 20.0, size)

# read frames
ret = True

while ret:
    ret, frame = video.read()

    if ret:
        # detect & track objects
        results = model_pretrained.track(frame, persist=True)

        # plot results
        composed = results[0].plot()

        # save video
        out.write(composed)

out.release()
video.release()

IndexError: list index out of range

![YOLOv8 License Plate Detection](https://github.com/ayush-raj13/anpr-and-parking-occupancy-analysis/blob/master/Vehicle%20Matching/outputs/uk_dash.webp?raw=1)

## Retraining YOLOv8

* [Dataset: Roboflow - License Plate Recognition Computer Vision Project](https://universe.roboflow.com/roboflow-universe-projects/license-plate-recognition-rxg4e)

Download the Dataset with YOLOv8 annotation and point YOLO to the `data.yaml` file that comes with the dataset:

```yml
train: ../train/images
val: ../valid/images
test: ../test/images

nc: 1
names: ['License_Plate']

roboflow:
  workspace: roboflow-universe-projects
  project: license-plate-recognition-rxg4e
  version: 4
  license: CC BY 4.0
  url: https://universe.roboflow.com/roboflow-universe-projects/license-plate-recognition-rxg4e/dataset/4
```

In [ ]:
# unzip downloaded dataset to `./datasets`
dataset = 'datasets/data.yaml'

# load a model
# backbone = YOLO("yolov8n.yaml")  # build a new model from scratch
backbone = YOLO("yolov8n.pt")  # load a pre-trained model (recommended for training)

In [ ]:
# Use the model
results = backbone.train(data=dataset, epochs=20)  # train the model

New https://pypi.org/project/ultralytics/8.0.176 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.173 🚀 Python-3.10.11 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6070MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets/data.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False

![YOLOv8 License Plate Detection](https://github.com/ayush-raj13/anpr-and-parking-occupancy-analysis/blob/master/Vehicle%20Matching/outputs/results.webp?raw=1)


| Class | Images | Instances | Box( P | R | mAP50 | mAP50-95) |
| -- |  -- |  -- | -- | -- | -- | -- |
| all |  2046 |  2132 | 0.986 | 0.954 | 0.984 | 0.701 |
| _Speed: 0.3ms preprocess, 5.1ms inference, 0.0ms loss, 0.5ms postprocess per image_ |
| _Model summary (fused): 168 layers, 3005843 parameters, 0 gradients_ |

In [ ]:
# Evaluate the model's performance on the validation set
results = backbone.val()

Ultralytics YOLOv8.0.173 🚀 Python-3.10.11 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6070MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients
val: Scanning /opt/app/03_object_detection_with_text_extraction_easyocr/datasets/valid/labels.cache... 2046 images, 3 backgrounds, 0 corrupt: 100%|████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:18<00:00,  6.74it/s]
                   all       2046       2132      0.986      0.954      0.984      0.701
Speed: 0.3ms preprocess, 5.7ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val


![YOLOv8 License Plate Detection](https://github.com/ayush-raj13/anpr-and-parking-occupancy-analysis/blob/master/Vehicle%20Matching/outputs/validation_batch.webp?raw=1)


| Class | Images | Instances | Box( P | R | mAP50 | mAP50-95) |
| -- |  -- |  -- | -- | -- | -- | -- |
| all |  2046 |  2132 | 0.986 | 0.954 | 0.984 | 0.701 |
| _Speed: 0.3ms preprocess, 5.7ms inference, 0.0ms loss, 0.6ms postprocess per image_ |

In [ ]:
# Perform object detection on an image using the model
results = backbone('inputs/cars.png')


image 1/1 /opt/app/03_object_detection_with_text_extraction_easyocr/inputs/cars.png: 384x640 2 License_Plates, 35.9ms
Speed: 1.7ms preprocess, 35.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
# Export the model to ONNX format
# success = model.export(imgsz=(640, 480), format='onnx', opset=12, optimize=False, half=False)
# Export to PyTorch format
success = backbone.export(imgsz=640, format='torchscript', optimize=False, half=False, int8=False)
# TorchScript: export success ✅ 1.5s, saved as 'runs/detect/train11/weights/best.torchscript' (11.9 MB)

Ultralytics YOLOv8.0.173 🚀 Python-3.10.11 torch-2.0.1 CPU (Intel Core(TM) i7-7700 3.60GHz)

PyTorch: starting from 'runs/detect/train11/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

TorchScript: starting export with torch 2.0.1...
TorchScript: export success ✅ 1.3s, saved as 'runs/detect/train11/weights/best.torchscript' (11.9 MB)

Export complete (2.6s)
Results saved to /opt/app/03_object_detection_with_text_extraction_easyocr/runs/detect/train11/weights
Predict:         yolo predict task=detect model=runs/detect/train11/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train11/weights/best.torchscript imgsz=640 data=datasets/data.yaml  
Visualize:       https://netron.app


In [ ]:
# pick pre-trained model
np_model = YOLO('runs/detect/train11/weights/best.torchscript')

In [ ]:
# read video by index
video = cv.VideoCapture(videos[1])
ret, frame = video.read()

# get video dims
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'DIVX')
out = cv.VideoWriter('./outputs/uk_dash_np_2.avi', fourcc, 20.0, size)

# read frames
ret = True

while ret:
    ret, frame = video.read()

    if ret:
        # detect & track objects
        results = np_model.track(frame, persist=True)

        # plot results
        composed = results[0].plot()

        # save video
        out.write(composed)

out.release()
video.release()


0: 640x640 2 License_Plates, 6.5ms
Speed: 5.7ms preprocess, 6.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 License_Plates, 6.4ms
Speed: 2.9ms preprocess, 6.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 License_Plates, 8.1ms
Speed: 2.0ms preprocess, 8.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 License_Plates, 6.6ms
Speed: 2.2ms preprocess, 6.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 License_Plates, 6.7ms
Speed: 2.2ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 License_Plates, 6.9ms
Speed: 2.0ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 License_Plates, 6.9ms
Speed: 2.5ms preprocess, 6.9ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 License_Plate, 7.9ms
Speed: 2.3ms preprocess, 7.9ms inference,

And now we have a model that is only interested in number plates:

![YOLOv8 License Plate Detection](https://github.com/ayush-raj13/anpr-and-parking-occupancy-analysis/blob/master/Vehicle%20Matching/outputs/uk_dash_np.webp?raw=1)

Though, the confusion matrix shows us that it also sees a lot of plates that do not exist - but at least it does not miss that many:

![YOLOv8 License Plate Detection](https://github.com/ayush-raj13/anpr-and-parking-occupancy-analysis/blob/master/Vehicle%20Matching/outputs/confusion_matrix_normalized.webp?raw=1)

## Improving Training Results

In [ ]:
# unzip downloaded dataset to `./datasets`
dataset = 'datasets/data.yaml'

# load a model
# backbone = YOLO("yolov8n.yaml")  # build a new model from scratch
backbone_small = YOLO("yolov8s.pt")  # load a pre-trained model (recommended for training)

In [ ]:
# Use the model
results_medium = backbone_small.train(data=dataset, epochs=100)  # train the model

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     57/100      4.57G     0.9052     0.3966      1.064          7        640: 100%|██████████| 1324/1324 [13:43<00:00,  1.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:26<00:00,  2.38it/s]
                   all       2046       2132      0.981      0.968      0.984      0.709

In [ ]:
# pick pre-trained model
np2_model = YOLO('runs/detect/train4/weights/best.pt')

In [ ]:
# Evaluate the model's performance on the validation set
results = np2_model.val()

Ultralytics YOLOv8.0.173 🚀 Python-3.10.11 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6070MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 2.35MB/s]
val: Scanning /opt/app/datasets/valid/labels.cache... 2046 images, 3 backgrounds, 0 corrupt: 100%|██████████| 2046/2046 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:35<00:00,  3.59it/s]
                   all       2046       2132      0.981      0.968      0.984       0.71
Speed: 0.3ms preprocess, 13.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val2
